In [1]:
! pip install transformers --quiet
! pip install datasets --quiet
! pip install trl --quiet #Transformer Reinforcement Learning
#! pip install peft
! pip install git+https://github.com/huggingface/peft.git --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from datasets import load_dataset
#dataset = load_dataset("tatsu-lab/alpaca", split="train")
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
#dataset[0]

In [3]:
open_qa = dataset.filter(lambda example: example['context'] == "")
print(len(dataset), len(open_qa))

15011 10544


In [4]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['response'][i]}"
        output_texts.append(text)
    return output_texts

In [5]:
from peft import PeftModel, PeftConfig, LoraConfig

In [6]:
'''
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

trainer = SFTTrainer(
    "EleutherAI/gpt-neo-125m",
    train_dataset=dataset,
    dataset_text_field="text",
    peft_config=peft_config
)'''

'\npeft_config = LoraConfig(\n    r=16,\n    lora_alpha=32,\n    lora_dropout=0.05,\n    bias="none",\n    task_type="CAUSAL_LM",\n)\n\ntrainer = SFTTrainer(\n    "EleutherAI/gpt-neo-125m",\n    train_dataset=dataset,\n    dataset_text_field="text",\n    peft_config=peft_config\n)'

In [7]:
'''
import gc
import torch
gc.collect()
torch.cuda.empty_cache()'
'''

"\nimport gc\nimport torch\ngc.collect()\ntorch.cuda.empty_cache()'\n"

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

base_model = "distilgpt2"
#base_model = "facebook/opt-350m"

model = AutoModelForCausalLM.from_pretrained(base_model)
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    #tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    #model.resize_token_embeddings(len(tokenizer))

Using pad_token, but it is not set yet.


In [31]:
input_text = tokenizer('What is meaning of life', return_tensors='pt')
output_ftm = model.generate(**input_text)
print(tokenizer.decode(output_ftm[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is meaning of life?
















In [ ]:
model_name = base_model.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-qna",
    evaluation_strategy = "no",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs = 1,
    push_to_hub=False,
    do_eval=False,
)

response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

trainer = SFTTrainer(
    model,
    train_dataset=open_qa,
    args=training_args,
    formatting_func=formatting_prompts_func,
    #peft_config=peft_config,
    data_collator=collator,
    max_seq_length=256,
)

trainer.train()

In [50]:
input_text = tokenizer('### Question: What is meaning of life', return_tensors='pt').to("cuda")
# input_text = tokenizer('what is life ?', return_tensors='pt').to("cuda")
output_ftm = model.generate(**input_text, max_new_tokens=20)
print(tokenizer.decode(output_ftm[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### Question: What is meaning of life?

Life is a process of living in a world that is not only a process of living
